In [48]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

# Load Dataset

In [2]:
def fetch_iwm_data():
    iwm = yf.Ticker("IWM")
    data = iwm.history(period="max")  # Adjust the period as needed
    return data

In [25]:
iwm_data = fetch_iwm_data()

In [26]:
iwm_data['Target'] = (iwm_data['High'] >= 1.01 * iwm_data['Open']).astype(int)

# Neural Network Model

In [5]:
# Function to create input features and target variable with sliding window of n days
def create_sliding_window(data, n):
    X, y = [], []
    for i in range(len(data) - n):
        X.append(data.iloc[i:i+n].drop('Target', axis=1).values)
        y.append(data.iloc[i+n]['Target'])
    return np.array(X), np.array(y)

In [6]:
# Define window size
n_days = 3

# Create sliding window dataset
X, y = create_sliding_window(iwm_data[['Open', 'Close', 'High', 'Low', 'Volume', 'Target']], n_days)


In [7]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [8]:
# Flatten the input data
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)


In [9]:
# Define the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/10
151/151 [==============================] - 2s 5ms/step - loss: 0.6253 - accuracy: 0.6864 - val_loss: 0.5980 - val_accuracy: 0.7030
Epoch 2/10
151/151 [==============================] - 1s 4ms/step - loss: 0.5956 - accuracy: 0.7022 - val_loss: 0.5921 - val_accuracy: 0.7063
Epoch 3/10
151/151 [==============================] - 1s 3ms/step - loss: 0.5918 - accuracy: 0.7002 - val_loss: 0.5913 - val_accuracy: 0.7080
Epoch 4/10
151/151 [==============================] - 1s 3ms/step - loss: 0.5855 - accuracy: 0.7008 - val_loss: 0.5865 - val_accuracy: 0.7146
Epoch 5/10
151/151 [==============================] - 0s 2ms/step - loss: 0.5852 - accuracy: 0.7016 - val_loss: 0.5815 - val_accuracy: 0.7072
Epoch 6/10
151/151 [==============================] - 0s 2ms/step - loss: 0.5819 - accuracy: 0.7060 - val_loss: 0.5777 - val_accuracy: 0.7155
Epoch 7/10
151/151 [==============================] - 0s 2ms/step - loss: 0.5816 - accuracy: 0.7047 - val_loss: 0.5822 - val_accuracy: 0.7097
Epoch 

In [11]:
# Predict probabilities for the test set
probabilities = model.predict(X_test_scaled)

# Threshold for entering a trade
threshold = 0.6

# Make trading decisions based on predicted probabilities
trades = []
for prob in probabilities:
    if prob > threshold:
        trades.append(1)  # Enter trade
    else:
        trades.append(0)  # Do not enter trade

38/38 [==============================] - 0s 1ms/step


In [12]:
y_test.shape

(1202,)

In [13]:
# Evaluate the model
accuracy = np.mean(trades == y_test)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.7004991680532446


In [14]:
probabilities

array([[0.25867563],
       [0.24384923],
       [0.27879897],
       ...,
       [0.2430691 ],
       [0.26635015],
       [0.16679505]], dtype=float32)

In [15]:
pip install fredapi

In [24]:
import pandas as pd
from fredapi import Fred

# Setup FRED API
api_key = ''  # Replace this with your actual FRED API key
fred = Fred(api_key=api_key)

# Function to fetch data from FRED
def fetch_fred_series(series_id, start_date, end_date):
    data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
    return data

# Load your ETF data
file_path = '/content/IWM_2001-2024.csv'
iwm_data = pd.read_csv(file_path)
iwm_data['Date'] = pd.to_datetime(iwm_data['Date'])

# Create a YearMonth column for merging
iwm_data['YearMonth'] = iwm_data['Date'].dt.to_period('M')

# Define the date range based on your dataset
start_date = iwm_data['Date'].min().strftime('%Y-%m-%d')
end_date = iwm_data['Date'].max().strftime('%Y-%m-%d')

# Fetch and prepare economic data
def prepare_and_fetch(series_id, value_name):
    data = fetch_fred_series(series_id, start_date, end_date)
    df = data.reset_index()
    df.columns = ['Date', value_name]
    df['YearMonth'] = pd.to_datetime(df['Date']).dt.to_period('M')
    return df[['YearMonth', value_name]]

# Prepare all dataframes
interest_rate_df = prepare_and_fetch('FEDFUNDS', 'Interest Rate')
cpi_df = prepare_and_fetch('CPIAUCSL', 'CPI')
gdp_df = prepare_and_fetch('GDP', 'GDP')
unemployment_df = prepare_and_fetch('UNRATE', 'Unemployment Rate')

# Merge all data into one DataFrame
from functools import reduce

dfs = [iwm_data, interest_rate_df, cpi_df, gdp_df, unemployment_df]
merged_data = reduce(lambda left, right: pd.merge(left, right, on='YearMonth', how='left'), dfs)

# Drop the YearMonth column if no longer needed
merged_data.drop(columns=['YearMonth'], inplace=True)

# Save the merged dataset
merged_data.to_csv('/content/merged_data.csv', index=False)

print(merged_data.head())


        Date       Open       High        Low      Close  Adj Close  Volume  \
0 2001-04-02  44.775002  45.000000  43.674999  43.750000  32.299583  340200   
1 2001-04-03  43.650002  43.650002  42.125000  42.264999  31.203249  507400   
2 2001-04-04  42.325001  42.924999  42.200001  42.400002  31.302916  356400   
3 2001-04-05  43.000000  44.450001  43.000000  44.450001  32.816372  950000   
4 2001-04-06  43.750000  43.889999  43.125000  43.200001  31.893551  405000   

   Interest Rate    CPI      GDP  Unemployment Rate  
0            4.8  176.4  10599.0                4.4  
1            4.8  176.4  10599.0                4.4  
2            4.8  176.4  10599.0                4.4  
3            4.8  176.4  10599.0                4.4  
4            4.8  176.4  10599.0                4.4  


In [28]:
merged_data['Target'] = (merged_data['High'] >= 1.01 * merged_data['Open']).astype(int)
merged_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Interest Rate,CPI,GDP,Unemployment Rate,Target
0,2001-04-02,44.775002,45.000000,43.674999,43.750000,32.299583,340200,4.8,176.4,10599.0,4.4,0
1,2001-04-03,43.650002,43.650002,42.125000,42.264999,31.203249,507400,4.8,176.4,10599.0,4.4,0
2,2001-04-04,42.325001,42.924999,42.200001,42.400002,31.302916,356400,4.8,176.4,10599.0,4.4,1
3,2001-04-05,43.000000,44.450001,43.000000,44.450001,32.816372,950000,4.8,176.4,10599.0,4.4,1
4,2001-04-06,43.750000,43.889999,43.125000,43.200001,31.893551,405000,4.8,176.4,10599.0,4.4,0


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Open'].plot(kind='hist', bins=20, title='Open')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['High'].plot(kind='hist', bins=20, title='High')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Low'].plot(kind='hist', bins=20, title='Low')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Date']
  ys = series['Close']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('Close')

from matplotlib import pyplot as plt
_df_12['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['Open'].plot(kind='line', figsize=(8, 4), title='Open')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['High'].plot(kind='line', figsize=(8, 4), title='High')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['Low'].plot(kind='line', figsize=(8, 4), title='Low')
plt.gca().spines[['top', 'right']].set_visible(False)

In [46]:
# Define window size
n_days = 3

# Create sliding window dataset
X, y = create_sliding_window(merged_data[['Open', 'Close', 'High', 'Low', 'Volume', 'Target', 'Interest Rate',	'CPI',	'GDP',	'Unemployment Rate']], n_days)


In [38]:
""
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [39]:
# Flatten the input data
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)


In [40]:
# Define the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/10
145/145 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 2/10
145/145 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 3/10
145/145 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 4/10
145/145 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 5/10
145/145 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 6/10
145/145 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 7/10
145/145 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7005 - val_loss: nan - val_accuracy: 0.6949
Epoch 8/10
145/145 [============================

In [42]:
# Predict probabilities for the test set
probabilities = model.predict(X_test_scaled)

# Threshold for entering a trade
threshold = 0.6

# Make trading decisions based on predicted probabilities
trades = []
for prob in probabilities:
    if prob > threshold:
        trades.append(1)  # Enter trade
    else:
        trades.append(0)  # Do not enter trade

37/37 [==============================] - 0s 1ms/step


In [43]:
y_test.shape

(1157,)

In [44]:
# Evaluate the model
accuracy = np.mean(trades == y_test)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.6949006050129646
